In [38]:
import os
import sys

import numpy as np
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
from matplotlib import colors
from hydra import initialize, compose
import pickle
from glob import glob
from pprint import pprint

# set paths to various directories based on the machine this code is being executed on
with initialize(version_base=None, config_path='config'):
    config = compose(config_name='config.yaml')  # overrides=['machine=uzay']

array_dir, data_dir, figure_dir, pickle_dir, repo_dir  = config.machine.array_dir, config.machine.data_dir, config.machine.figure_dir, config.machine.pickle_dir, config.machine.repo_dir

# enable use of local modules
if repo_dir not in sys.path:
    sys.path.append(repo_dir)

# set matplotlib style
plt.style.use(f'{repo_dir}/mejiro/mplstyle/science.mplstyle')

from mejiro.lenses.test import SampleStrongLens
from mejiro.plots import diagnostic_plot, plot, plot_util, overplot
from mejiro.analysis import stats
from mejiro.utils import util
from mejiro.helpers import pyhalo

In [39]:
pipeline_params = util.hydra_to_dict(config.pipeline)

In [40]:
bands = pipeline_params['bands']

In [41]:
bands = bands * 3
print(bands)

['F087', 'F106', 'F129', 'F158', 'F184', 'F087', 'F106', 'F129', 'F158', 'F184', 'F087', 'F106', 'F129', 'F158', 'F184']


In [42]:
results = [np.ones((45, 45))] * len(set(bands)) * 3

In [43]:
print(set(bands))
print(len(set(bands)))

{'F158', 'F087', 'F129', 'F184', 'F106'}
5


In [44]:
j = 0
for i, (band, result) in enumerate(zip(bands, results)):
    if j == 0:
        print('array')
    elif j == 1:
        print('lens')
    elif j == 2:
        print('source')
    if i % len(set(bands)) == len(set(bands)) - 1:
        j += 1
    # np.save(os.path.join(output_dir, f'galsim_{lens.uid}_{band}.npy'), result)

array
array
array
array
array
lens
lens
lens
lens
lens
source
source
source
source
source
